In [ ]:
import scipy.io
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import os
import math
import time

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [ ]:
def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1

    return patchesData, patchesLabels.astype("int")

In [ ]:
dataset = ['India', 'Pavia'][1]
random_split = True

In [ ]:
if dataset == 'India':
    raw_data = scipy.io.loadmat('./drive/MyDrive/HSI-datasets/indian_pines_corrected.mat')
    disjoint_data = scipy.io.loadmat(os.getcwd() + '/drive/MyDrive/HSI-datasets/indianpines_disjoint_dset.mat')
    all_labels = scipy.io.loadmat(os.getcwd() + '/drive/MyDrive/HSI-datasets/indian_pines_gt.mat')

    X_all = raw_data['indian_pines_corrected']
    y_disjoint = disjoint_data['indianpines_disjoint_dset']
    y_all = all_labels['indian_pines_gt']

    test_ratio = 0.45
else:
    raw_data = scipy.io.loadmat('./drive/MyDrive/HSI-datasets/paviaU.mat')
    disjoint_data = scipy.io.loadmat(os.getcwd() + '/drive/MyDrive/HSI-datasets/TRpavia_fixed.mat')
    all_labels = scipy.io.loadmat(os.getcwd() + '/drive/MyDrive/HSI-datasets/paviaU_gt.mat')

    X_all = raw_data['paviaU']
    y_disjoint = disjoint_data['TRpavia_fixed']
    y_all = all_labels['paviaU_gt']

    test_ratio = 0.93

In [ ]:
import scipy.io
import numpy as np
import copy
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# -*- coding: utf-8 -*-
# Torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim
from torch.nn import init
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
from torchsummary import summary
import torch.quantization
from torch.quantization import QuantStub, DeQuantStub

In [ ]:
cuda_device = torch.device("cuda:0")
cpu_device = torch.device("cpu:0")

In [ ]:
num_channels = X_all.shape[-1]
W = X_all.shape[0]
H = X_all.shape[1]
num_components = 40
window = 19

In [ ]:
X_all = X_all.reshape(W*H, num_channels)
y_disjoint = y_disjoint.reshape(-1, 1).flatten()

X_all = X_all.astype(np.float32)

In [ ]:
pca = PCA(n_components=num_components)
X_all = pca.fit_transform(X_all)

X_all = X_all.reshape(W, H, num_components)

In [ ]:
X, y = createImageCubes(X_all, y_all, windowSize=window, removeZeroLabels = False)

In [ ]:
np.save('data.npy', X)
np.save('labels.npy', y)

In [ ]:
del X_all, y_all, X, y, raw_data

In [ ]:
X = np.load('data.npy', mmap_mode='c')
y = np.load('labels.npy', mmap_mode='c')

In [ ]:
X = np.transpose(X, (0, 3, 1, 2))

In [ ]:
X.shape

(207400, 40, 19, 19)

In [ ]:
if random_split:
    nonzero_idx = np.where(y != 0)
    X = X[nonzero_idx]
    y = y[nonzero_idx]
    y = y - 1
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=42)
else:
    train_idx = np.where((y_disjoint != 0) & (y != 0))
    X_train = X[train_idx]
    y_train = y[train_idx]

    test_idx = np.where((y_disjoint == 0) & (y != 0))
    X_test = X[test_idx]
    y_test = y[test_idx]

    y_train -= 1
    y_test -= 1

    print('Disjoint')

In [ ]:
X_train = X_train.reshape(X_train.shape[0], num_components*window*window)
X_test = X_test.reshape(X_test.shape[0], num_components*window*window)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(X_train.shape[0], num_components, window, window)
X_test = X_test.reshape(X_test.shape[0], num_components, window, window)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
X_test.shape

(39782, 40, 19, 19)

In [ ]:
tensor_x = torch.Tensor(X_train) # transform to torch tensor
tensor_y = torch.Tensor(y_train)
tensor_y = tensor_y.type(torch.LongTensor)

train_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
train_dataloader = DataLoader(train_dataset, batch_size=100) # create your dataloader

In [ ]:
tensor_x = torch.Tensor(X_val) # transform to torch tensor
tensor_y = torch.Tensor(y_val)
tensor_y = tensor_y.type(torch.LongTensor)

val_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
val_dataloader = DataLoader(val_dataset, batch_size=100) # create your dataloader

In [ ]:
tensor_x = torch.Tensor(X_test) # transform to torch tensor
tensor_y = torch.Tensor(y_test)
tensor_y = tensor_y.type(torch.LongTensor)

test_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
test_dataloader = DataLoader(test_dataset, batch_size=100) # create your dataloader

In [ ]:
class cnn2d(nn.Module):
    """
    CNN2d network
    """

    @staticmethod
    def weight_init(m):
        if isinstance(m, nn.Conv2d):
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            m.weight.data.normal_(0, math.sqrt(2. / n))
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(0.5)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.01)
            m.bias.data.zero_()

    def __init__(self, input_channels, n_classes):
        super(cnn2d, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=50, kernel_size=(5, 5))
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_channels=50, out_channels=100, kernel_size=(5, 5))
        self.relu2 = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d((2, 2))

        self.fc1 = nn.Linear(2500, 100)
        self.relu3 = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(100, n_classes)

        self.apply(self.weight_init)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.pool(self.relu2(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def evaluate_model(model, test_loader, device, criterion=None):

    model.eval()
    model.to(device)

    running_loss = 0
    running_corrects = 0

    for inputs, labels in test_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        if criterion is not None:
            loss = criterion(outputs, labels).item()
        else:
            loss = 0

        # statistics
        running_loss += loss * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    eval_loss = running_loss / len(test_loader.dataset)
    eval_accuracy = running_corrects / len(test_loader.dataset)

    return eval_loss, eval_accuracy

In [ ]:
def train_model(model, train_loader, test_loader, epochs, lr, device, save_file):

    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.02)
    criterion = nn.CrossEntropyLoss()
    min_valid_loss = np.inf

    for epoch in range(epochs):

        # Training
        model.train()

        running_loss = 0
        running_corrects = 0

        for inputs, labels in train_loader:

            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = running_corrects / len(train_loader.dataset)

        # Evaluation
        model.eval()
        eval_loss, eval_accuracy = evaluate_model(model=model, test_loader=test_loader, device=device, criterion=criterion)

        # Save model.
        if min_valid_loss > (eval_loss):
            min_valid_loss = (eval_loss)
            save_model(model=model, model_dir="./", model_filename=save_file)

        if epoch % (epochs//10) == 0 or epoch == (epochs-1):
            print("Epoch: {:02d} Train Loss: {:.3f} Train Acc: {:.3f} Eval Loss: {:.3f} Eval Acc: {:.3f}".format(epoch, \
            train_loss, train_accuracy, eval_loss, eval_accuracy))

    return model

In [ ]:
def calibrate_model(model, loader, device=torch.device("cpu:0")):

    model.to(device)
    model.eval()

    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        _ = model(inputs)

In [ ]:
def measure_inference_latency(model,
                              device,
                              input_size=(1, 40, 19, 19),
                              num_samples=100,
                              num_warmups=10):

    model.to(device)
    model.eval()

    x = torch.rand(size=input_size).to(device)

    with torch.no_grad():
        for _ in range(num_warmups):
            _ = model(x)
    torch.cuda.synchronize()

    with torch.no_grad():
        start_time = time.time()
        for _ in range(num_samples):
            _ = model(x)
            torch.cuda.synchronize()
        end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_time_ave = elapsed_time / num_samples

    return elapsed_time_ave

In [ ]:
def save_model(model, model_dir, model_filename):

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.save(model.state_dict(), model_filepath)

def load_model(model, model_filepath, device):

    model.load_state_dict(torch.load(model_filepath, map_location=device))

    return model

In [ ]:
def save_torchscript_model(model, model_dir, model_filename):

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.jit.save(torch.jit.script(model), model_filepath)

def load_torchscript_model(model_filepath, device):

    model = torch.jit.load(model_filepath, map_location=device)

    return model

In [ ]:
def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (MB):', size/1e6)
    os.remove('temp.p')
    return size

In [ ]:
class QuantizedCNN(nn.Module):
    def __init__(self, model_fp32):
        super(QuantizedCNN, self).__init__()
        # QuantStub converts tensors from floating point to quantized.
        # This will only be used for inputs.
        self.quant = torch.quantization.QuantStub()
        # DeQuantStub converts tensors from quantized to floating point.
        # This will only be used for outputs.
        self.dequant = torch.quantization.DeQuantStub()
        # FP32 model
        self.model_fp32 = model_fp32

    def forward(self, x):
        # manually specify where tensors will be converted from floating
        # point to quantized in the quantized model
        x = self.quant(x)
        x = self.model_fp32(x)
        # manually specify where tensors will be converted from quantized
        # to floating point in the quantized model
        x = self.dequant(x)
        return x

In [ ]:
def model_equivalence(model_1, model_2, device, rtol=1e-05, atol=1e-08, num_tests=100, input_size=(1,40,19,19)):

    model_1.to(device)
    model_2.to(device)

    for _ in range(num_tests):
        x = torch.rand(size=input_size).to(device)
        y1 = model_1(x).detach().cpu().numpy()
        y2 = model_2(x).detach().cpu().numpy()
        if np.allclose(a=y1, b=y2, rtol=rtol, atol=atol, equal_nan=False) == False:
            print("Model equivalence test sample failed: ")
            print(y1)
            print(y2)
            return False

    return True

In [ ]:
model = cnn2d(num_components, len(np.unique(y_train)))

In [ ]:
model = train_model(model=model, train_loader=train_dataloader, test_loader=val_dataloader, \
                    epochs=100, lr=1e-3, device=cuda_device, save_file="best_model.pth")

Epoch: 00 Train Loss: 1.406 Train Acc: 0.507 Eval Loss: 0.963 Eval Acc: 0.640
Epoch: 10 Train Loss: 0.035 Train Acc: 0.994 Eval Loss: 0.042 Eval Acc: 0.990
Epoch: 20 Train Loss: 0.025 Train Acc: 0.997 Eval Loss: 0.062 Eval Acc: 0.987
Epoch: 30 Train Loss: 0.015 Train Acc: 1.000 Eval Loss: 0.033 Eval Acc: 0.990
Epoch: 40 Train Loss: 0.028 Train Acc: 0.994 Eval Loss: 0.086 Eval Acc: 0.977
Epoch: 50 Train Loss: 0.014 Train Acc: 1.000 Eval Loss: 0.036 Eval Acc: 0.990
Epoch: 60 Train Loss: 0.014 Train Acc: 1.000 Eval Loss: 0.034 Eval Acc: 0.987
Epoch: 70 Train Loss: 0.020 Train Acc: 0.996 Eval Loss: 0.050 Eval Acc: 0.980
Epoch: 80 Train Loss: 0.012 Train Acc: 1.000 Eval Loss: 0.031 Eval Acc: 0.990
Epoch: 90 Train Loss: 0.016 Train Acc: 0.999 Eval Loss: 0.048 Eval Acc: 0.980
Epoch: 99 Train Loss: 0.012 Train Acc: 1.000 Eval Loss: 0.028 Eval Acc: 0.990


# **Dynamic Quantization**

In [ ]:
# Load a pretrained model.
model = load_model(model=model, model_filepath="./best_model.pth", device=cuda_device)
model.to(cpu_device)

cnn2d(
  (conv1): Conv2d(40, 50, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(50, 100, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU(inplace=True)
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2500, out_features=100, bias=True)
  (relu3): ReLU(inplace=True)
  (fc2): Linear(in_features=100, out_features=9, bias=True)
)

In [ ]:
quantized_model = torch.quantization.quantize_dynamic(
    model, {nn.Conv2d, nn.Linear, nn.ReLU}, dtype=torch.qint8
)

In [ ]:
_, fp32_eval_accuracy = evaluate_model(model=model, test_loader=test_dataloader, device=cpu_device, criterion=None)
_, int8_eval_accuracy = evaluate_model(model=quantized_model, test_loader=test_dataloader, device=cpu_device, criterion=None)

print("FP32 evaluation accuracy: {:.3f}".format(fp32_eval_accuracy))
print("INT8 evaluation accuracy: {:.3f}".format(int8_eval_accuracy))

FP32 evaluation accuracy: 0.996
INT8 evaluation accuracy: 0.996


In [ ]:
# compare the sizes
f=print_size_of_model(model,"fp32")
q=print_size_of_model(quantized_model,"int8")
print("{0:.2f} times smaller".format(f/q))

model:  fp32  	 Size (MB): 1.707121
model:  int8  	 Size (MB): 0.955863
1.79 times smaller


In [ ]:
fp32_cpu_inference_latency = measure_inference_latency(model=model, device=cpu_device, input_size=(1,num_components, window, window), num_samples=100)
int8_cpu_inference_latency = measure_inference_latency(model=quantized_model, device=cpu_device, input_size=(1,num_components, window, window), num_samples=100)

print("FP32 CPU Inference Latency: {:.2f} ms / sample".format(fp32_cpu_inference_latency * 1000))
print("INT8 CPU Inference Latency: {:.2f} ms / sample".format(int8_cpu_inference_latency * 1000))

FP32 CPU Inference Latency: 1.05 ms / sample
INT8 CPU Inference Latency: 1.05 ms / sample


# **Static Quantization**

In [ ]:
# Load a pretrained model.
model = load_model(model=model, model_filepath="./best_model.pth", device=cuda_device)
# Move the model to CPU since static quantization does not support CUDA currently.
model.to(cpu_device)
# Make a copy of the model for layer fusion
fused_model = copy.deepcopy(model)

In [ ]:
model.eval()
# The model has to be switched to evaluation mode before any layer fusion.
# Otherwise the quantization will not work correctly.
fused_model.eval()

# Fuse the model in place rather manually.
fused_model = torch.quantization.fuse_modules(fused_model, [["conv1", "relu1"],
                                                            ["conv2", "relu2"],
                                                            ["fc1", "relu3"]], inplace=True)

In [ ]:
# Print FP32 model.
print(model)
# Print fused model.
print(fused_model)

cnn2d(
  (conv1): Conv2d(40, 50, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(50, 100, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU(inplace=True)
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2500, out_features=100, bias=True)
  (relu3): ReLU(inplace=True)
  (fc2): Linear(in_features=100, out_features=9, bias=True)
)
cnn2d(
  (conv1): ConvReLU2d(
    (0): Conv2d(40, 50, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (relu1): Identity()
  (conv2): ConvReLU2d(
    (0): Conv2d(50, 100, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (relu2): Identity()
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): LinearReLU(
    (0): Linear(in_features=2500, out_features=100, bias=True)
    (1): ReLU(inplace=True)
  )
  (relu3): Identity()
  (fc2): Linear(in_features=100, out_features=9

In [ ]:
# Model and fused model should be equivalent.
assert model_equivalence(model_1=model, model_2=fused_model, device=cpu_device, rtol=1e-03, \
                         atol=1e-06, num_tests=100, input_size=(1,num_components, window, window)),\
                         "Fused model is not equivalent to the original model!"

In [ ]:
# Prepare the model for static quantization. This inserts observers in
# the model that will observe activation tensors during calibration.
quantized_model = QuantizedCNN(model_fp32=fused_model)
# Select quantization schemes from 
# https://pytorch.org/docs/stable/quantization-support.html
quantization_config = torch.quantization.get_default_qconfig("fbgemm")

In [ ]:
quantized_model.qconfig = quantization_config
    
# Print quantization configurations
print(quantized_model.qconfig)

torch.quantization.prepare(quantized_model, inplace=True)

# Use training data for calibration.
calibrate_model(model=quantized_model, loader=train_dataloader, device=cpu_device)

quantized_model = torch.quantization.convert(quantized_model, inplace=True)

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.HistogramObserver'>, reduce_range=True){}, weight=functools.partial(<class 'torch.ao.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric){})


/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [ ]:
# Using high-level static quantization wrapper
quantized_model.eval()

# Print quantized model.
print(quantized_model)

# Save quantized model.
save_torchscript_model(model=quantized_model, model_dir="./", model_filename="quantized_model.pth")

QuantizedCNN(
  (quant): Quantize(scale=tensor([0.3600]), zero_point=tensor([53]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (model_fp32): cnn2d(
    (conv1): QuantizedConvReLU2d(40, 50, kernel_size=(5, 5), stride=(1, 1), scale=0.11217733472585678, zero_point=0)
    (relu1): Identity()
    (conv2): QuantizedConvReLU2d(50, 100, kernel_size=(5, 5), stride=(1, 1), scale=0.28110361099243164, zero_point=0)
    (relu2): Identity()
    (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (fc1): QuantizedLinearReLU(in_features=2500, out_features=100, scale=0.26749753952026367, zero_point=0, qscheme=torch.per_channel_affine)
    (relu3): Identity()
    (fc2): QuantizedLinear(in_features=100, out_features=9, scale=0.3958379030227661, zero_point=83, qscheme=torch.per_channel_affine)
  )
)


In [ ]:
# Load quantized model.
quantized_jit_model = load_torchscript_model(model_filepath="./quantized_model.pth", device=cpu_device)

_, fp32_eval_accuracy = evaluate_model(model=model, test_loader=test_dataloader, device=cpu_device, criterion=None)
_, int8_eval_accuracy = evaluate_model(model=quantized_model, test_loader=test_dataloader, device=cpu_device, criterion=None)

print("FP32 evaluation accuracy: {:.3f}".format(fp32_eval_accuracy))
print("INT8 evaluation accuracy: {:.3f}".format(int8_eval_accuracy))

FP32 evaluation accuracy: 0.996
INT8 evaluation accuracy: 0.996


In [ ]:
fp32_cpu_inference_latency = measure_inference_latency(model=model, device=cpu_device, input_size=(1,num_components, window, window), num_samples=100)
int8_cpu_inference_latency = measure_inference_latency(model=quantized_model, device=cpu_device, input_size=(1,num_components, window, window), num_samples=100)
int8_jit_cpu_inference_latency = measure_inference_latency(model=quantized_jit_model, device=cpu_device, input_size=(1,num_components, window, window), num_samples=100)
fp32_gpu_inference_latency = measure_inference_latency(model=model, device=cuda_device, input_size=(1,num_components, window, window), num_samples=100)

print("FP32 CPU Inference Latency: {:.2f} ms / sample".format(fp32_cpu_inference_latency * 1000))
print("FP32 CUDA Inference Latency: {:.2f} ms / sample".format(fp32_gpu_inference_latency * 1000))
print("INT8 CPU Inference Latency: {:.2f} ms / sample".format(int8_cpu_inference_latency * 1000))
print("INT8 JIT CPU Inference Latency: {:.2f} ms / sample".format(int8_jit_cpu_inference_latency * 1000))

FP32 CPU Inference Latency: 0.96 ms / sample
FP32 CUDA Inference Latency: 0.44 ms / sample
INT8 CPU Inference Latency: 0.68 ms / sample
INT8 JIT CPU Inference Latency: 0.52 ms / sample


In [ ]:
# compare the sizes
f=print_size_of_model(model,"fp32")
q=print_size_of_model(quantized_model,"int8")
print("{0:.2f} times smaller".format(f/q))

model:  fp32  	 Size (MB): 1.707249
model:  int8  	 Size (MB): 0.438659
3.89 times smaller


# **Quantization Aware Training (QAT)**

In [ ]:
# Load a pretrained model.
model = load_model(model=model, model_filepath="./best_model.pth", device=cuda_device)
# Move the model to CPU since static quantization does not support CUDA currently.
model.to(cpu_device)
# Make a copy of the model for layer fusion
fused_model = copy.deepcopy(model)

In [ ]:
model.train()
# The model has to be switched to training mode before any layer fusion.
# Otherwise the quantization aware training will not work correctly.
fused_model.train()

cnn2d(
  (conv1): Conv2d(40, 50, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(50, 100, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU(inplace=True)
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2500, out_features=100, bias=True)
  (relu3): ReLU(inplace=True)
  (fc2): Linear(in_features=100, out_features=9, bias=True)
)

In [ ]:
# Fuse the model in place rather manually.
fused_model = torch.quantization.fuse_modules(fused_model, [["conv1", "relu1"],
                                                            ["conv2", "relu2"],
                                                            ["fc1", "relu3"]], inplace=True)

In [ ]:
# Print FP32 model.
print(model)
# Print fused model.
print(fused_model)

cnn2d(
  (conv1): Conv2d(40, 50, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(50, 100, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU(inplace=True)
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2500, out_features=100, bias=True)
  (relu3): ReLU(inplace=True)
  (fc2): Linear(in_features=100, out_features=9, bias=True)
)
cnn2d(
  (conv1): ConvReLU2d(
    (0): Conv2d(40, 50, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (relu1): Identity()
  (conv2): ConvReLU2d(
    (0): Conv2d(50, 100, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (relu2): Identity()
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): LinearReLU(
    (0): Linear(in_features=2500, out_features=100, bias=True)
    (1): ReLU(inplace=True)
  )
  (relu3): Identity()
  (fc2): Linear(in_features=100, out_features=9

In [ ]:
# Model and fused model should be equivalent.
# Model and fused model should be equivalent.
model.eval()
fused_model.eval()
assert model_equivalence(model_1=model, model_2=fused_model, device=cpu_device, rtol=1e-03, \
                         atol=1e-06, num_tests=100, input_size=(1,num_components, window, window)),\
                         "Fused model is not equivalent to the original model!"

In [ ]:
# Prepare the model for static quantization. This inserts observers in
# the model that will observe activation tensors during calibration.
quantized_model = QuantizedCNN(model_fp32=fused_model)
# Select quantization schemes from 
# https://pytorch.org/docs/stable/quantization-support.html
quantization_config = torch.quantization.get_default_qconfig("fbgemm")

In [ ]:
quantized_model.qconfig = quantization_config
    
# Print quantization configurations
print(quantized_model.qconfig)

torch.quantization.prepare_qat(quantized_model, inplace=True)

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.HistogramObserver'>, reduce_range=True){}, weight=functools.partial(<class 'torch.ao.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric){})


QuantizedCNN(
  (quant): QuantStub(
    (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
  )
  (dequant): DeQuantStub()
  (model_fp32): cnn2d(
    (conv1): ConvReLU2d(
      40, 50, kernel_size=(5, 5), stride=(1, 1)
      (weight_fake_quant): PerChannelMinMaxObserver(min_val=tensor([]), max_val=tensor([]))
      (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
    )
    (relu1): Identity()
    (conv2): ConvReLU2d(
      50, 100, kernel_size=(5, 5), stride=(1, 1)
      (weight_fake_quant): PerChannelMinMaxObserver(min_val=tensor([]), max_val=tensor([]))
      (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
    )
    (relu2): Identity()
    (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (fc1): LinearReLU(
      in_features=2500, out_features=100, bias=True
      (weight_fake_quant): PerChannelMinMaxObserver(min_val=tensor([]), max_val=tensor([]))
      (activation_post_

In [ ]:
# # Use training data for calibration.
print("Training QAT Model...")
quantized_model.train()
train_model(model=quantized_model, train_loader=train_dataloader, test_loader=val_dataloader, \
            epochs=100, lr=1e-5, device=cuda_device, save_file="qat_model.pth")
quantized_model.to(cpu_device)

Training QAT Model...
Epoch: 00 Train Loss: 0.009 Train Acc: 1.000 Eval Loss: 0.022 Eval Acc: 0.993
Epoch: 10 Train Loss: 0.009 Train Acc: 1.000 Eval Loss: 0.024 Eval Acc: 0.993
Epoch: 20 Train Loss: 0.010 Train Acc: 1.000 Eval Loss: 0.026 Eval Acc: 0.990
Epoch: 30 Train Loss: 0.010 Train Acc: 1.000 Eval Loss: 0.027 Eval Acc: 0.990
Epoch: 40 Train Loss: 0.011 Train Acc: 1.000 Eval Loss: 0.027 Eval Acc: 0.990
Epoch: 50 Train Loss: 0.011 Train Acc: 1.000 Eval Loss: 0.028 Eval Acc: 0.990
Epoch: 60 Train Loss: 0.011 Train Acc: 1.000 Eval Loss: 0.029 Eval Acc: 0.990
Epoch: 70 Train Loss: 0.011 Train Acc: 1.000 Eval Loss: 0.029 Eval Acc: 0.990
Epoch: 80 Train Loss: 0.011 Train Acc: 1.000 Eval Loss: 0.029 Eval Acc: 0.990
Epoch: 90 Train Loss: 0.011 Train Acc: 1.000 Eval Loss: 0.030 Eval Acc: 0.990
Epoch: 99 Train Loss: 0.011 Train Acc: 1.000 Eval Loss: 0.030 Eval Acc: 0.990


QuantizedCNN(
  (quant): QuantStub(
    (activation_post_process): HistogramObserver(min_val=-51.8465461730957, max_val=55.524837493896484)
  )
  (dequant): DeQuantStub()
  (model_fp32): cnn2d(
    (conv1): ConvReLU2d(
      40, 50, kernel_size=(5, 5), stride=(1, 1)
      (weight_fake_quant): PerChannelMinMaxObserver(
        min_val=tensor([-0.0203, -0.0091, -0.0382, -0.0292, -0.0437, -0.0423, -0.0455, -0.0029,
                -0.0425, -0.0339, -0.0325, -0.0018, -0.0034, -0.0419, -0.0523, -0.0217,
                -0.0033, -0.0137, -0.0512, -0.0333, -0.0083, -0.0444, -0.0088, -0.0056,
                -0.0042, -0.0101, -0.0739, -0.0115, -0.0835, -0.0034, -0.0234, -0.0059,
                -0.0398, -0.0708, -0.0105, -0.0622, -0.0508, -0.0027, -0.0052, -0.0030,
                -0.0160, -0.0128, -0.0139, -0.0067, -0.0121, -0.0102, -0.0041, -0.0299,
                -0.0272, -0.0051]), max_val=tensor([0.0139, 0.0099, 0.0341, 0.0495, 0.0531, 0.0213, 0.0331, 0.0040, 0.0452,
                0.01

In [ ]:
quantized_model = torch.quantization.convert(quantized_model, inplace=True)

In [ ]:
# Using high-level static quantization wrapper
quantized_model.eval()

# Print quantized model.
print(quantized_model)

# Save quantized model.
save_torchscript_model(model=quantized_model, model_dir="./", model_filename="qat_model.pth")

QuantizedCNN(
  (quant): Quantize(scale=tensor([0.3596]), zero_point=tensor([52]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (model_fp32): cnn2d(
    (conv1): QuantizedConvReLU2d(40, 50, kernel_size=(5, 5), stride=(1, 1), scale=0.10366735607385635, zero_point=0)
    (relu1): Identity()
    (conv2): QuantizedConvReLU2d(50, 100, kernel_size=(5, 5), stride=(1, 1), scale=0.2531552016735077, zero_point=0)
    (relu2): Identity()
    (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (fc1): QuantizedLinearReLU(in_features=2500, out_features=100, scale=0.25121748447418213, zero_point=0, qscheme=torch.per_channel_affine)
    (relu3): Identity()
    (fc2): QuantizedLinear(in_features=100, out_features=9, scale=0.37192127108573914, zero_point=83, qscheme=torch.per_channel_affine)
  )
)


In [ ]:
# Load quantized model.
quantized_jit_model = load_torchscript_model(model_filepath="./qat_model.pth", device=cpu_device)

_, fp32_eval_accuracy = evaluate_model(model=model, test_loader=test_dataloader, device=cpu_device, criterion=None)
_, int8_eval_accuracy = evaluate_model(model=quantized_jit_model, test_loader=test_dataloader, device=cpu_device, criterion=None)

print("FP32 evaluation accuracy: {:.3f}".format(fp32_eval_accuracy))
print("INT8 evaluation accuracy: {:.3f}".format(int8_eval_accuracy))

FP32 evaluation accuracy: 0.996
INT8 evaluation accuracy: 0.996


In [ ]:
fp32_cpu_inference_latency = measure_inference_latency(model=model, device=cpu_device, input_size=(1,num_components, window, window), num_samples=100)
int8_cpu_inference_latency = measure_inference_latency(model=quantized_model, device=cpu_device, input_size=(1,num_components, window, window), num_samples=100)
int8_jit_cpu_inference_latency = measure_inference_latency(model=quantized_jit_model, device=cpu_device, input_size=(1,num_components, window, window), num_samples=100)
fp32_gpu_inference_latency = measure_inference_latency(model=model, device=cuda_device, input_size=(1,num_components, window, window), num_samples=100)

print("FP32 CPU Inference Latency: {:.2f} ms / sample".format(fp32_cpu_inference_latency * 1000))
print("FP32 CUDA Inference Latency: {:.2f} ms / sample".format(fp32_gpu_inference_latency * 1000))
print("INT8 CPU Inference Latency: {:.2f} ms / sample".format(int8_cpu_inference_latency * 1000))
print("INT8 JIT CPU Inference Latency: {:.2f} ms / sample".format(int8_jit_cpu_inference_latency * 1000))

FP32 CPU Inference Latency: 1.08 ms / sample
FP32 CUDA Inference Latency: 0.45 ms / sample
INT8 CPU Inference Latency: 0.73 ms / sample
INT8 JIT CPU Inference Latency: 0.46 ms / sample


In [ ]:
# compare the sizes
f=print_size_of_model(model,"fp32")
q=print_size_of_model(quantized_model,"int8")
print("{0:.2f} times smaller".format(f/q))

model:  fp32  	 Size (MB): 1.707249
model:  int8  	 Size (MB): 0.438659
3.89 times smaller
